# JULIA MPI First Example: pi computaton

First step was to load MPI on my mac.  Seems mpich and openmpi are two reasonable choices
with probably no beginner's reason to prefer one over the other. <br>

I did  <t> brew install gcc </t> first to get the gcc compiler.  I ran into problems.  
The magic thing that told me what to do was <t> brew doctor </t>.  It wanted me to type
<t> xcode-select --install </t> and when I did, all was good.  I then typed
<t> brew install mpich </t> and mpi was just working.

My first example was to reproduce <a href="http://www.mcs.anl.gov/research/projects/mpi/tutorial/mpiexmpl/src/pi/C/solution.html">
the classic mypi </a> in the notebook

In [ ]:
Pkg.add("MPI")

In [1]:
using MPI

In [2]:
m = MPIManager(np=4)

MPI.MPIManager(np=4,launched=false,mode=MPI_ON_WORKERS)

In [3]:
addprocs(m)
#@mpi_do m comm = MPI.COMM_WORLD

4-element Array{Int64,1}:
 2
 3
 4
 5

In [4]:
@mpi_do m comm = MPI.COMM_WORLD
#
# Enter number of intervals, and tell every processor
# Traditional MPI would do this with a BCAST
 @mpi_do m n = 10

In [5]:
# Let's see if the processors got it
@mpi_do m println(n)

	From worker 2:	10
	From worker 3:	10
	From worker 4:	10
	From worker 5:	10


In [6]:
# my MPI id
@mpi_do m myid = MPI.Comm_rank(comm)
@mpi_do m println(myid)

	From worker 4:	2
	From worker 5:	3
	From worker 2:	0
	From worker 3:	1


In [7]:
# Get the number of processors
@mpi_do m np = MPI.Comm_size(comm)
@mpi_do m println(np)

	From worker 3:	4
	From worker 4:	4
	From worker 2:	4
	From worker 5:	4


Compute $\int_0^1 4/(1+x^2) dx= 4 atan(x)]_0^1$ which evaluates to $\pi$

In [8]:
@time @mpi_do m begin
    n = 50_000_000
    comm = MPI.COMM_WORLD
    s = 0.0
    for i = MPI.Comm_rank(comm) + 1 : MPI.Comm_size(comm) : n 
        x = (i - .5)/n 
        s += 4/(1 + x^2) 
    end
    mypi = s/n
    our_π = MPI.Reduce(mypi, MPI.SUM, 0, comm)
    if myid==0
        println(our_π - π) 
    end
end

	From worker 2:	1.1146639167236572e-13
  7.635322 seconds (3.49 k allocations: 226.630 KiB)


In [10]:
[( @fetchfrom i π-4*mypi, π  )for i in workers()]

4-element Array{Tuple{Float64,Irrational{:π}},1}:
 (-6.0e-8, π = 3.1415926535897...)    
 (-2.00002e-8, π = 3.1415926535897...)
 (1.99998e-8, π = 3.1415926535897...) 
 (5.99999e-8, π = 3.1415926535897...) 

In [24]:
function f_serial()
    n = 50_000_000
    h = 1/n
    our_π = 0.0
    for i = 0:h:1
        our_π += 4/(1 + i^2)
    end
    our_π*h
end

function f_serial2(n) # or we can make n a variable
    our_π = 0.0
    h=1/n
    for i = 1:n
        x = (i - 0.5)*h
        our_π += 4/(1 + x^2)
    end
    our_π/n
end

f_serial2 (generic function with 1 method)

In [23]:
f_serial() #warmup
f_serial()
f_serial2(50_000_000) #warmup
@time f_serial()
@time f_serial2(50_000_000)

  0.222391 seconds (5 allocations: 176 bytes)
  0.217645 seconds (5 allocations: 176 bytes)


3.1415926535895617

In [25]:
using Interact

In [36]:
println(Float64(π))
@manipulate for n = [1000, 10_000, 100_000, 1_000_000, 10_000_000]
  f_serial2(n)
end


Interact.Options{:ToggleButtons,Any}(37: "input-10" = 1000 Any , "n", 1000, "1000", 0, Interact.OptionDict(DataStructures.OrderedDict{Any,Any}("1000"=>1000,"10000"=>10000,"100000"=>100000,"1000000"=>1000000,"10000000"=>10000000), Dict{Any,Any}(Pair{Any,Any}(1000000, "1000000"),Pair{Any,Any}(10000, "10000"),Pair{Any,Any}(100000, "100000"),Pair{Any,Any}(1000, "1000"),Pair{Any,Any}(10000000, "10000000"))), Any[], Any[], true, "horizontal")

3.141592653589793


3.1415927369231227

In [38]:
function f_par(n)

 @mpi_do m begin
    comm = MPI.COMM_WORLD
        
       
    s = 0.0
    for i = MPI.Comm_rank(comm) + 1 : MPI.Comm_size(comm) : $n 
        x = (i - .5)/$n 
        s += 4/(1 + x^2) 
    end
    mypi = s/$n
    our_π = MPI.Reduce(mypi, MPI.SUM, 0, comm)
    #if myid==0
     #   println(our_π - π) 
   # end
end
@fetchfrom 2 our_π   
end

f_par (generic function with 1 method)

In [39]:
@mpi_do m function _pi_sum_par(n)
    comm = MPI.COMM_WORLD

    s = 0.0
    for i = MPI.Comm_rank(comm) + 1 : MPI.Comm_size(comm) : n
        x = (i - .5)/n 
        s += 4/(1 + x^2) 
    end
    mypi = s/n
    our_π = MPI.Reduce(mypi, MPI.SUM, 0, comm)
    return our_π
end

function f_par2(n)
    @mpi_do m _pi_sum_par($n)
    @fetchfrom 2 our_π
end

f_par(50_000_000) #warmup
f_par(50_000_000)
f_par2(50_000_000) #warmup
@time f_par2(50_000_000)

  0.112648 seconds (587 allocations: 40.344 KiB)


3.1415926535899046

In [ ]:
π

In [40]:
[f_par(10^k) for k=3:9]-π

7-element Array{Float64,1}:
  8.33333e-8 
  8.33331e-10
  8.32312e-12
  1.10134e-13
 -1.07025e-13
  4.23661e-13
 -2.4869e-14 

In [41]:
using DistributedArrays

In [1]:
Pkg.build("Elemental")

INFO: Building Elemental
CMake Warning at CMakeLists.txt:250 (message):
  Build mode not specified, defaulting to Release build.


CMake Error at cmake/external_projects/ElMath.cmake:292 (message):
  daxpy_64_ was not detected
Call Stack (most recent call first):
  CMakeLists.txt:358 (include)


==============================[ ERROR: Elemental ]==============================

LoadError: failed process: Process(`cmake -D CMAKE_INSTALL_PREFIX=/Users/alanedelman/.julia/v0.6/Elemental/deps/usr -D INSTALL_PYTHON_PACKAGE=OFF -D PYTHON_EXECUTABLE= -D PYTHON_SITE_PACKAGES= -D EL_USE_64BIT_INTS=ON -D EL_USE_64BIT_BLAS_INTS=ON -D MATH_LIBS=/Applications/Julia-0.6.app/Contents/Resources/julia/lib/julia/libopenblas64_.dylib -D EL_BLAS_SUFFIX=_64_ -D EL_LAPACK_SUFFIX=_64_ -D CMAKE_INSTALL_RPATH=/Users/alanedelman/.julia/v0.6/Elemental/deps/usr/lib /Users/alanedelman/.julia/v0.6/Elemental/deps/src/Elemental`, ProcessExited(1)) [1]
while loading /Users/alanedelman/.julia/v0.6/Elemental/deps/build.jl,

In [42]:
using Elemental

LoadError: LoadError: LoadError: [91merror compiling ElIntType: could not load library "/Users/alanedelman/.julia/v0.6/Elemental/deps/usr/lib/libEl"
dlopen(/Users/alanedelman/.julia/v0.6/Elemental/deps/usr/lib/libEl.dylib, 1): image not found[39m
while loading /Users/alanedelman/.julia/v0.6/Elemental/src/core/types.jl, in expression starting on line 8
while loading /Users/alanedelman/.julia/v0.6/Elemental/src/Elemental.jl, in expression starting on line 46

In [ ]:
A = drandn(1000, 800);

In [ ]:
svdvals(A)